# adtools.sandbox.decorators 教程：装饰器用法

除了直接使用 `SandboxExecutor` 类，`adtools` 还提供了一个更简便的装饰器 `@sandbox_run`，可以直接将函数或类方法标记为在沙盒中执行。这使得代码更加简洁和Pythonic。

## 引入装饰器

首先，我们需要从 `adtools.sandbox` 导入 `sandbox_run`。


In [ ]:
from adtools.sandbox import sandbox_run
import time


## 装饰普通函数

我们可以直接装饰一个独立的函数。调用该函数时，它将在单独的进程中运行。


In [ ]:

@sandbox_run(timeout=2.0)
def expensive_calculation(n):
    print(f"开始计算 {n} 的平方（在子进程中）...")
    time.sleep(1)  # 模拟耗时操作
    return n * n

print("调用函数...")
# 注意：返回值是一个包含执行结果元数据的字典
result = expensive_calculation(10)

print(f"完整结果: {result}")
print(f"计算结果: {result['result']}")
print(f"耗时: {result['evaluate_time']:.4f}s")


## 处理超时

如果函数执行时间超过了 `timeout` 参数指定的时间，沙盒将强制终止该进程，并返回 `None` 作为结果。


In [ ]:

@sandbox_run(timeout=1.0)
def slow_function():
    time.sleep(2)
    return "Finished"

print("调用超时函数...")
result = slow_function()
print(f"结果: {result['result']} (应为 None)")
print(f"错误信息: {result['error_msg']}")


## 装饰类方法

`@sandbox_run` 同样适用于类的方法。它会自动处理 `self` 参数，在沙盒中创建实例的副本并执行方法。


In [ ]:

class DataProcessor:
    def __init__(self, data):
        self.data = data

    @sandbox_run(timeout=5.0)
    def process(self):
        # 在沙盒中，self.data 是可访问的
        return sum(self.data)

processor = DataProcessor([1, 2, 3, 4, 5])
print("调用类方法...")
result = processor.process()
print(f"处理结果: {result['result']}")


## 使用 Ray 后端

你可以通过设置 `sandbox_type='ray'` 来使用 Ray 进行执行。这需要确保 Ray 已安装并可能需要初始化。

```python
# 确保已安装 ray: pip install ray

@sandbox_run(sandbox_type='ray', timeout=5.0)
def ray_function(x):
    return x + 1

# result = ray_function(10)
```


## 参数详解

- `sandbox_type`: 沙盒类型，可选 `'process'` (默认) 或 `'ray'`。
- `timeout`: 超时时间（秒）。如果为 `None` 则不限制。
- `redirect_to_devnull`: 如果为 `True`，则将子进程的 stdout/stderr 重定向到 `/dev/null`，防止输出污染控制台。
- `**executor_init_kwargs`: 传递给 `SandboxExecutor` 初始化的其他参数，例如 `debug_mode=True`。
